In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/LICENSE
/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/README
/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/maestro-v2.0.0.csv
/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/maestro-v2.0.0.json
/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/2017/MIDI-Unprocessed_058_PIANO058_MID--AUDIO-split_07-07-17_Piano-e_2-02_wav--3.mp3
/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/2017/MIDI-Unprocessed_062_PIANO062_MID--AUDIO-split_07-07-17_Piano-e_2-07_wav--2.midi
/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/2017/MIDI-Unprocessed_065_PIANO065_MID--AUDIO-split_07-07-17_Piano-e_3-01_wav--2.mp3
/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/2017/MIDI-Unprocessed_060_PIANO060_MID--AUDIO-split_07-07-17_Piano-e_2-04_wav--4.midi
/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/2017/MIDI-Unprocessed_073_PIANO073_MID--AUDIO-split_07-08-17_Piano-e_2-02_wav--4.mp3
/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/2017/MIDI-Unprocessed_054_PIA

In [2]:
!pip install note_seq
!pip install librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 5.8 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26118 sha256=62cbbcce19a963c6be3e203962874d2901032caf0e81d8bfaaeaab57eef848ec
  Stored in directory: /root/.cache/pip/wheels/01/94/38/6fe5b0e582dca953d1ef8f2e8714fa1def8e5bf514c25c11fb
  Created wheel for pretty-midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592305 sha256=cc0ddd4d0a9ecdc927b6b1d2dade3dc1c636259a04631a8436b60744d78d2c99
  Stored in directory: /root/.cache/pip/wheels/ea/8a/5a/615ed5b0cab54051df

In [3]:
ds=pd.read_csv('/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/maestro-v2.0.0.csv')
ds.head()


,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Alban Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160
1,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
2,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
3,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
4,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508


In [4]:
ds.shape

(1282, 7)

In [5]:
ds.columns

Index(['canonical_composer', 'canonical_title', 'split', 'year',
       'midi_filename', 'audio_filename', 'duration'],
      dtype='object')

In [6]:
sorted_df = ds.sort_values(by=['split', 'year'])
sorted_df

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
56,Claude Debussy,"Etude No. 7 ""Study in Chromatic Steps""",test,2004,2004/MIDI-Unprocessed_SMF_17_R1_2004_03-06_ORI...,2004/MIDI-Unprocessed_SMF_17_R1_2004_03-06_ORI...,134.376478
123,Domenico Scarlatti,"Sonata, K239",test,2004,2004/MIDI-Unprocessed_XP_04_R1_2004_03-05_ORIG...,2004/MIDI-Unprocessed_XP_04_R1_2004_03-05_ORIG...,196.015447
485,Franz Schubert / Franz Liszt,"Song Transcriptions: Aufenthalt, Gretchen am S...",test,2004,2004/MIDI-Unprocessed_XP_18_R1_2004_01-02_ORIG...,2004/MIDI-Unprocessed_XP_18_R1_2004_01-02_ORIG...,189.921448
487,Franz Schubert / Franz Liszt,"Song Transcriptions: Aufenthalt, Gretchen am S...",test,2004,2004/MIDI-Unprocessed_XP_18_R1_2004_01-02_ORIG...,2004/MIDI-Unprocessed_XP_18_R1_2004_01-02_ORIG...,249.113581
762,Johann Sebastian Bach,"Prelude and Fugue in B-Moll, No. 22 WTKII",test,2004,2004/MIDI-Unprocessed_XP_03_R1_2004_01-02_ORIG...,2004/MIDI-Unprocessed_XP_03_R1_2004_01-02_ORIG...,420.449614
...,...,...,...,...,...,...,...
1015,Ludwig van Beethoven,"Sonata No. 29 in B-flat Major, Op. 106",validation,2018,2018/MIDI-Unprocessed_Recital1-3_MID--AUDIO_02...,2018/MIDI-Unprocessed_Recital1-3_MID--AUDIO_02...,2457.807371
1165,Robert Schumann,"Sonata No. 2 in G Minor, Op. 22",validation,2018,2018/MIDI-Unprocessed_Recital4_MID--AUDIO_04_R...,2018/MIDI-Unprocessed_Recital4_MID--AUDIO_04_R...,976.820604
1200,Sergei Rachmaninoff,Etudes-Tableaux Op. 33,validation,2018,2018/MIDI-Unprocessed_Recital5-7_MID--AUDIO_05...,2018/MIDI-Unprocessed_Recital5-7_MID--AUDIO_05...,1355.377918
1222,Sergei Rachmaninoff,Prelude Op. 3 No. 2 in C-sharp Minor,validation,2018,2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_...,2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_...,239.852858


In [7]:
import pandas as pd

# Sort DataFrame on 'split' column first, then 'year' column
df_sorted = ds.sort_values(by=['split', 'year'])

# Group all rows where 'year' is '2014' together
df_grouped = df_sorted.groupby('year').apply(lambda x: x[x['year'] == '2014'].append(x[x['year'] != '2014']))
df_grouped


canonical_composer  \
year                                      
2004 56                  Claude Debussy   
     123             Domenico Scarlatti   
     485   Franz Schubert / Franz Liszt   
     487   Franz Schubert / Franz Liszt   
     762          Johann Sebastian Bach   
...                                 ...   
2018 1015          Ludwig van Beethoven   
     1165               Robert Schumann   
     1200           Sergei Rachmaninoff   
     1222           Sergei Rachmaninoff   
     1280       Wolfgang Amadeus Mozart   

                                             canonical_title       split  \
year                                                                       
2004 56               Etude No. 7 "Study in Chromatic Steps"        test   
     123                                        Sonata, K239        test   
     485   Song Transcriptions: Aufenthalt, Gretchen am S...        test   
     487   Song Transcriptions: Aufenthalt, Gretchen am S...        test   
     762           Prelude and Fugue in B-Moll, No. 22 WTKII        test   
...                                                      ...         ...   
2018 1015             Sonata No. 29 in B-flat Major, Op. 106  validation   
     1165                    Sonata No. 2 in G Minor, Op. 22  validation   
     1200                             Etudes-Tableaux Op. 33  validation   
     1222               Prelude Op. 3 No. 2 in C-sharp Minor  validation   
     1280                       Sonata in F Major, K533/K494  validation   

           year                                      midi_filename  \
year                                                                 
2004 56    2004  2004/MIDI-Unprocessed_SMF_17_R1_2004_03-06_ORI...   
     123   2004  2004/MIDI-Unprocessed_XP_04_R1_2004_03-05_ORIG...   
     485   2004  2004/MIDI-Unprocessed_XP_18_R1_2004_01-02_ORIG...   
     487   2004  2004/MIDI-Unprocessed_XP_18_R1_2004_01-02_ORIG...   
     762   2004  2004/MIDI-Unprocessed_XP_03_R1_2004_01-02_ORIG...   
...         ...                                                ...   
2018 1015  2018  2018/MIDI-Unprocessed_Recital1-3_MID--AUDIO_02...   
     1165  2018  2018/MIDI-Unprocessed_Recital4_MID--AUDIO_04_R...   
     1200  2018  2018/MIDI-Unprocessed_Recital5-7_MID--AUDIO_05...   
     1222  2018  2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_...   
     1280  2018  2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_...   

                                              audio_filename     duration  
year                                                                       
2004 56    2004/MIDI-Unprocessed_SMF_17_R1_2004_03-06_ORI...   134.376478  
     123   2004/MIDI-Unprocessed_XP_04_R1_2004_03-05_ORIG...   196.015447  
     485   2004/MIDI-Unprocessed_XP_18_R1_2004_01-02_ORIG...   189.921448  
     487   2004/MIDI-Unprocessed_XP_18_R1_2004_01-02_ORIG...   249.113581  
     762   2004/MIDI-Unprocessed_XP_03_R1_2004_01-02_ORIG...   420.449614  
...                                                      ...          ...  
2018 1015  2018/MIDI-Unprocessed_Recital1-3_MID--AUDIO_02...  2457.807371  
     1165  2018/MIDI-Unprocessed_Recital4_MID--AUDIO_04_R...   976.820604  
     1200  2018/MIDI-Unprocessed_Recital5-7_MID--AUDIO_05...  1355.377918  
     1222  2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_...   239.852858  
     1280  2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_...  1068.751602  

[1282 rows x 7 columns]

In [8]:
pd.set_option("display.max.rows",None)
sort=ds.sort_values('year')
sort

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
1281,Wolfgang Amadeus Mozart,"Twelve Variations, K179 (189a)",train,2004,2004/MIDI-Unprocessed_XP_04_R1_2004_01-02_ORIG...,2004/MIDI-Unprocessed_XP_04_R1_2004_01-02_ORIG...,630.266303
708,Isaac Albéniz,"Iberia, Evocation, Triana",train,2004,2004/MIDI-Unprocessed_XP_04_R1_2004_03-05_ORIG...,2004/MIDI-Unprocessed_XP_04_R1_2004_03-05_ORIG...,291.849566
123,Domenico Scarlatti,"Sonata, K239",test,2004,2004/MIDI-Unprocessed_XP_04_R1_2004_03-05_ORIG...,2004/MIDI-Unprocessed_XP_04_R1_2004_03-05_ORIG...,196.015447
707,Isaac Albéniz,"Iberia, Evocation, Triana",train,2004,2004/MIDI-Unprocessed_XP_04_R1_2004_03-05_ORIG...,2004/MIDI-Unprocessed_XP_04_R1_2004_03-05_ORIG...,361.027781
697,George Frideric Handel,Suite No. 3 in D Minor,train,2004,2004/MIDI-Unprocessed_XP_08_R1_2004_01-02_ORIG...,2004/MIDI-Unprocessed_XP_08_R1_2004_01-02_ORIG...,720.089756
688,Frédéric Chopin,"Sonata in B Minor, Op. 58",train,2004,2004/MIDI-Unprocessed_XP_18_R1_2004_04_ORIG_MI...,2004/MIDI-Unprocessed_XP_18_R1_2004_04_ORIG_MI...,1442.054507
681,Frédéric Chopin,"Sonata No. 3 in B Minor, Op. 58",train,2004,2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MI...,2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MI...,768.135555
680,Frédéric Chopin,"Sonata No. 3 in B Minor, Op. 58",train,2004,2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MI...,2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MI...,154.986722
679,Frédéric Chopin,"Sonata No. 3 in B Minor, Op. 58",train,2004,2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MI...,2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MI...,493.142776
665,Frédéric Chopin,"Scherzo No. 4 in E Major, Op. 54",train,2004,2004/MIDI-Unprocessed_XP_10_R1_2004_03-04_ORIG...,2004/MIDI-Unprocessed_XP_10_R1_2004_03-04_ORIG...,637.803674


In [9]:
ds.groupby('year').sum()

,duration
year,
2004,83447.026930
2006,90998.442655
2008,52653.903008
2009,82836.857601
2011,56123.393396
2013,51666.629690
2014,89381.173973
2015,51445.037500
2017,57875.593545


In [10]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1282 entries, 0 to 1281
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   canonical_composer  1282 non-null   object 
 1   canonical_title     1282 non-null   object 
 2   split               1282 non-null   object 
 3   year                1282 non-null   int64  
 4   midi_filename       1282 non-null   object 
 5   audio_filename      1282 non-null   object 
 6   duration            1282 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 70.2+ KB


In [11]:
ds.describe()

,year,duration
count,1282.000000,1282.000000
mean,2011.333853,565.034568
std,4.367139,447.298440
min,2004.000000,45.155208
25%,2008.000000,262.703213
50%,2011.000000,433.190117
75%,2015.000000,686.831559
max,2018.000000,2624.663508


In [12]:
ds.describe(include=object)

,canonical_composer,canonical_title,split,midi_filename,audio_filename
count,1282,1282,1282,1282,1282
unique,61,857,3,1282,1282
top,Frédéric Chopin,"Thirty-Two Variations in C Minor, WoO 80",train,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...
freq,201,18,967,1,1


In [13]:
import json
import pandas as pd
import plotly_express as px
def get_json_file(path):
    f = open(path)
    return json.load(f)
metadata_dict = get_json_file('/kaggle/input/themaestrodatasetv2/maestro-v2.0.0/maestro-v2.0.0.json')
metadata_dict[0]


{'canonical_composer': 'Alban Berg',
 'canonical_title': 'Sonata Op. 1',
 'split': 'train',
 'year': 2018,
 'midi_filename': '2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi',
 'audio_filename': '2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.wav',
 'duration': 698.661160312}